# Managing Models & Releases

The Bailo python client enables intuitive interaction with the Bailo service, from within a python environment. This example notebook will run through the following concepts:

* Creating a new model on Bailo.
* Creating and populating a model card.
* Retrieving models from the service.
* Making changes to the model, and model card.
* Creating and managing specific releases, with files.

Prerequisites:

* Python 3.8.1 or higher (including a notebook environment for this demo).
* A local or remote Bailo service (see https://github.com/gchq/Bailo).


## Introduction

The Bailo python client is split into two sub-packages: **core** and **helper**.

* **Core:** For direct interactions with the service endpoints.
* **Helper:** For more intuitive interactions with the service, using classes (e.g. Model) to handle operations.

In order to create helper classes, you will first need to instantiate a `Client()` object from the core. By default, this object will not support any authentication. However, Bailo also supports PKI authentication, which you can use from Python by passing a `PkiAgent()` object into the `Client()` object when you instantiate it.

In [2]:
# Necessary import statements

from bailo import Model, Client

# Instantiating the PkiAgent(), if using.
# agent = PkiAgent(cert='', key='', auth='')

# Instantiating the Bailo client

client = Client("http://127.0.0.1:8080") # <- INSERT BAILO URL (if not hosting locally)


## Creating a new model

### Creating and updating the base model

In this section, we'll create a new model using the `Model.create()` classmethod. On the Bailo service, a model must consist of at least 4 parameters upon creation. These are **name**, **description**, **visibility** and **team_id**. Other attributes like model cards, files, or releases are added later on. Below, we use the `Client()` object created before when instantiating the new `Model()` object. 

NOTE: This creates the model on your Bailo service too! The `model_id` is assigned by the backend, and we will use this later to retrieve the model.

In [3]:
model = Model.create(client=client, name="YOLOv5", description="YOLOv5 model for object detection.", team_id="uncategorised")

model_id = model.model_id

You may make changes to these attributes and then call the `update()` method to relay the changes to the service, as below:

```python
model.name = "New Name"
model.update()
```


### Creating and populating a model card

When creating a model card, first we need to generate an empty one using the `card_from_schema()` method. In this instance, we will use **minimal-general-v10-beta**. You can manage custom schemas using the `Schema()` helper class, but this is out of scope for this demo.

In [4]:
model.card_from_schema(schema_id='minimal-general-v10-beta')

model.model_card_version

1

If successful, the above will have created a new model card, and the `model_card_version` attribute should be set to 1.

Next, we can populate the model card using the `update_model_card()` method. This can be used any time you want to make changes, and the backend will create a new model card version each time. We'll learn how to retrieve model cards later (either the latest, or a specific release).

NOTE: Your model card must match the schema, otherwise an error will be thrown.

In [5]:
new_card = {
  'overview': {
    'tags': [],
    'modelSummary': 'YOLOv5 model card for demonstration purposes.',
  }
}

model.update_model_card(model_card=new_card)

model.model_card_version

2

If successful, the `model_card_version` will now be 2!

## Retrieving an existing model

### Using the .from_id() method

In this section, we'll retrieve our previous model using the `Model.from_id()` classmethod. This will create your `Model()` object as before, but using existing information retrieved from the service.

In [6]:
model = Model.from_id(client=client, model_id=model_id)

model.description

'YOLOv5 model for object detection.'

If successful, the model description we set earlier should be displayed above.

## Creating and managing releases for models

On the Bailo service, different versions of the same model are managed using **releases**. Generally, this is for code changes and minor adjustments that don't drastically change the behaviour of a model. In this section we will create a **release** and upload a file.

### Creating a release

`Release()` is a separate helper class in itself, but we can use our `Model()` object to create and retrieve releases. Running the below code will create a new release of the model, and return an instantiated `Release()` object which we will use to upload files with.

In [7]:
release_one = model.create_release(version='1.0.0', notes='Note')

### Uploading files to a release

To upload files for a release, we can use the release `upload()` method which will take a file name, and a `BytesIO` type containing the file contents. Using the **demo_file.txt** file, or any local file, would look as follows.

NOTE: The `upload()` method takes a `BytesIO` type to allow for other integrations, such as with S3 or Weights & Biases.

In [8]:
print(release_one.model_id)
print(model.model_id)

with open("demo_file.txt") as f:
    release_one.upload("demo_file.txt", f)

yolov5-wb9i5i
yolov5-wb9i5i


### Downloading files from release

Similarly you can also download files from release using a response object. Using the same release you can view the contents of **demo_file** as follows.

In [9]:
res = release_one.download("demo_file.txt")

print(res.content)

b'This is a demo file.\n'


## Retrieving a release

We can retrieve the latest release for a model using the model `get_latest_release()` method. Alternatively, we can retrieve a specific release using the model `get_release()` method. Both of these will return an instantiated `Release()` object.

In [10]:
release_latest = model.get_latest_release()
release_one = model.get_release(version='1.0.0')

#To demonstrate this is the same release:
if release_latest == release_one:
    print("Successfully retrieved identical releases!")


Successfully retrieved identical releases!
